In [38]:
using CSV
using DataFrames
using Distributions

# Define the distributions
d = Uniform(0, 2000)
d1 = Uniform(0, 100)
i = 10  # Number of products
t = 12  # Number of time periods

# Generate random coefficients
ci = rand(d)
cr = rand(d)
co = rand(d)
cu = rand(d)
cs = rand(d)

# Initialize W[0] and I[0]
W_init = round(Int, rand(d))
I_init = round(Int, rand(d))
B_init = round(Int, rand(d))

# Define a parameter K
K = rand(d1)

# Define constants
cf = 173490
ch = 2406.6
cb = 0.15 * cr
T = 12 * rand(2:5)

# Generate an array of random values for D
D = [round(Int, rand(d)) for _ in 1:t, _ in 1:i ]

# Create a DataFrame for the demand D with columns as time periods and rows as products
df_demand = DataFrame(D, :auto)

# Create a DataFrame for the other parameters
df_params = DataFrame(
    ci = ci,
    cr = cr,
    co = co,
    cu = cu,
    cs = cs,
    W_init = W_init,
    I_init = I_init,
    K = K,
    T = T,
    cb = cb,
    B_init = B_init
)

# Save the DataFrame to a CSV file
CSV.write("data8.csv", df_params)
CSV.write("demand_data8.csv", df_demand)

# Print the DataFrame to verify the format
println(df_params)
println(df_demand)


1×11 DataFrame
 Row │ ci       cr       co       cu       cs       W_init  I_init  K        T      cb       B_init 
     │ Float64  Float64  Float64  Float64  Float64  Int64   Int64   Float64  Int64  Float64  Int64  
─────┼──────────────────────────────────────────────────────────────────────────────────────────────
   1 │ 1213.22  438.237  1555.37  587.519  179.623     685     612  33.4792     36  65.7356     754
12×10 DataFrame
 Row │ x1     x2     x3     x4     x5     x6     x7     x8     x9     x10   
     │ Int64  Int64  Int64  Int64  Int64  Int64  Int64  Int64  Int64  Int64 
─────┼──────────────────────────────────────────────────────────────────────
   1 │   598    952   1273    780    627    289    311   1856   1594      1
   2 │  1690    502    181   1662    287   1574    953   1002   1759   1532
   3 │   820   1819   1865   1680    420   1503    574    299     82   1391
   4 │  1956    714   1295    434   1241   1925   1641    917   1479   1861
   5 │  1039   1131      8    5

In [36]:
using JuMP
using Gurobi
using Distributions
using CSV
using DataFrames

# Load the CSV file into a DataFrame
demand_data = CSV.read(raw"C:\Users\Admin\Downloads\demand_data7.csv", DataFrame)
para_data = CSV.read(raw"C:\Users\Admin\Downloads\data7.csv", DataFrame)

# Extract the relevant data from the DataFrame
#for i in 1:n
    #D[i] = data.D[i]
#end
ci = para_data.ci[1]
cr = para_data.cr[1]
co = para_data.co[1]
cu = para_data.cu[1]
cs = para_data.cs[1]
cb = para_data.cb[1]
W_init = para_data.W_init[1]
I_init = para_data.I_init[1]
K = para_data.K[1]
T = para_data.T[1]
B_init = para_data.B_init[1]
# Define the distributions
#d = Uniform(0, 100000)
#d1 = Uniform(0, 100)

# Extract the demand data and convert it to a 2D array (transposed)
n_products = size(demand_data, 2) 
n_periods = size(demand_data, 1) # Assuming the first column is 'Product'
D = Array{Float64}(undef, n_periods, n_products)

for t in 1:12
    for i in 1:10
        D[t, i] = demand_data[t, i]  # Adjust for 1-based indexing in Julia
    end
end

In [15]:
D[1,2]

13712.0

In [16]:
n_products

10

In [20]:
n_periods = size(demand_data, 1) 

12

In [26]:
n_periods

12

In [35]:
m = Model(Gurobi.Optimizer)
n = T
nt = 20
k_max = 10
j_max = 10

# Define the variables
@variable(m, W[0:n, 1:k_max, 1:j_max] >= 0, Int)
@variable(m, H[1:n, 1:k_max, 1:j_max] >= 0, Int)
@variable(m, F[1:n, 1:k_max, 1:j_max] >= 0, Int)
@variable(m, I[0:n, 1:k_max, 1:j_max] >= 0)
@variable(m, P[1:n, 1:k_max, 1:j_max] >= 0)
@variable(m, O[1:n, 1:k_max, 1:j_max] >= 0)
@variable(m, U[1:n, 1:k_max, 1:j_max] >= 0)
@variable(m, S[1:n, 1:k_max, 1:j_max] >= 0)
@variable(m, B[0:n, 1:k_max, 1:j_max] >= 0, Int)

# Add initial constraints for W[0] and I[0]
for k in 1:k_max
    for j in 1:j_max
        @constraint(m, W[0, k, j] == W_init)
        @constraint(m, I[0, k, j] == I_init)
        @constraint(m, B[0, k, j] == B_init)
    end
end

# Define the constraints
for i in 1:n
    for k in 1:k_max
        for j in 1:j_max
            @constraint(m, W[i, k, j] == W[i-1, k, j] + H[i, k, j] - F[i, k, j])
            @constraint(m, P[i, k, j] == K * nt * W[i, k, j] + O[i, k, j] - U[i, k, j])
            @constraint(m, I[i, k, j] - B[i, k, j] == I[i-1, k, j] - B[i-1, k, j] + P[i, k, j] + S[i, k, j] - D[i])
            @constraint(m, H[i, k, j] <= 0.1 * W[i, k, j])                     
            @constraint(m, F[i, k, j] <= 0.1 * W[i, k, j])
        end
    end
end

# Define the objective function
@objective(m, Min,
    ch * sum(H[i, k, j] for i in 1:n, k in 1:k_max, j in 1:j_max) +
    cf * sum(F[i, k, j] for i in 1:n, k in 1:k_max, j in 1:j_max) +
    ci * sum(I[i, k, j] for i in 1:n, k in 1:k_max, j in 1:j_max) +
    co * sum(O[i, k, j] for i in 1:n, k in 1:k_max, j in 1:j_max) +
    cu * sum(U[i, k, j] for i in 1:n, k in 1:k_max, j in 1:j_max) +
    cs * sum(S[i, k, j] for i in 1:n, k in 1:k_max, j in 1:j_max) +
    cr * sum(P[i, k, j] for i in 1:n, k in 1:k_max, j in 1:j_max) +
    cb * sum(B[i, k, j] for i in 1:n, k in 1:k_max, j in 1:j_max)
)

# Optimize the model
optimize!(m)

# Print the results
for i in 1:n
    for k in 1:k_max
        for j in 1:j_max
            println("W[$i][$k][$j] = ", value(W[i, k, j]))
            println("H[$i][$k][$j] = ", value(H[i, k, j]))
            println("F[$i][$k][$j] = ", value(F[i, k, j]))
            println("I[$i][$k][$j] = ", value(I[i, k, j]))
            println("P[$i][$k][$j] = ", value(P[i, k, j]))
            println("O[$i][$k][$j] = ", value(O[i, k, j]))
            println("U[$i][$k][$j] = ", value(U[i, k, j]))
            println("S[$i][$k][$j] = ", value(S[i, k, j]))
            println("B[$i][$k][$j] = ", value(B[i, k, j]))
        end
    end
end

10682.0